In [1]:
include("build_profiles.jl")

build_ngram_profiles (generic function with 1 method)

In [4]:

lang = "kab"
    D1 = TatoebaDataset("corpus/tatoeba", "tatoeba_train.txt", langs=[lang])
    D2 = WikiDataSet("corpus/wikipedia/train", langs=[lang])
    println("# ", lang)

# kab


In [20]:
Ds = [D1, D2]
ngram = 7
ratio = 0.9
minfreq = 10
maxsize = 100000
Ds = count_dataset_all_ngrams.([D1, D2], ngram)

2-element Vector{Dict{Vector{UInt8}, Float32}}:
 Dict([0x8d, 0x61, 0x6d, 0x20, 0x79, 0x61, 0x67] => 1.0, [0x69, 0x66, 0x65, 0x6d] => 111.0, [0x74, 0x6e, 0x69, 0x6b, 0x20] => 3.0, [0x64, 0x64, 0x20, 0x75, 0x6c, 0x61, 0x63] => 1.0, [0x67, 0x75, 0x6d, 0x65, 0x6d, 0x74, 0x20] => 1.0, [0x20, 0x69, 0x7a, 0x72, 0x65, 0xc9] => 35.0, [0x70, 0x61, 0x68, 0x69, 0x79] => 1.0, [0x61, 0x20, 0x73, 0x77, 0x61, 0x64] => 1.0, [0x8d, 0x20, 0x75, 0x71, 0x65, 0xe1, 0xb9] => 1.0, [0x75, 0x72, 0x20, 0x74, 0x20, 0x64, 0x65] => 1.0…)
 Dict([0x74, 0x6e, 0x69, 0x6b, 0x20] => 1.0, [0x61, 0x6e, 0x6e, 0x61, 0x20, 0x66] => 1.0, [0x75, 0x67, 0x75, 0x73, 0x74, 0x65] => 2.0, [0x20, 0x69, 0x74, 0x20, 0x6c, 0x77, 0x61] => 2.0, [0x61, 0x6c, 0x69, 0x77, 0x65, 0x6e] => 1.0, [0x61, 0x64, 0x20, 0x6e, 0x20, 0x63] => 2.0, [0x72, 0x65, 0x67, 0x74, 0x20, 0x75] => 1.0, [0x74, 0x6d, 0x61, 0x72, 0x74, 0x72] => 1.0, [0x61, 0x66, 0x61, 0x20, 0x61, 0x6c] => 3.0, [0x61, 0x20, 0x73, 0x77, 0x61, 0x64] => 1.0…)

In [6]:

    S = mergewith(+, Ds...)
    tokens_list = []
    vocab_list = []
    for D in Ds
        tokens = sum(v for (k, v) in D if length(k) == 1)
        push!(tokens_list, tokens)
        push!(vocab_list, length(D))
        sc = log(tokens) / tokens
        for (k, v) in D
            D[k] *= sc
        end
    end
    G = mergewith(+, Ds...)
    sG = sort(collect(G), by=i -> (-last(i), length(first(i)), first(i)))
    cp1 = cutoff_at_ratio(sG, ratio)
    cp2 = findfirst(k -> S[k] < minfreq, first.(sG)) - 1
    cp = min(cp1, cp2, maxsize)
    println("total vocab: $(length(S)/1000)k; tokens: $(sum(vocab_list)/1e6)M; dataset ratio: $(vocab_list[1:end]/sum(vocab_list))(vocab) $(tokens_list/sum(tokens_list))(token)")
    println("Profile size: $(cp/1000)k($(round(cp/length(sG)*100))%) @freq=$(S[sG[cp][1]]); [min($cp1, $cp2)]")
    sG[1:cp], count_ngrams(sG)

total vocab: 2172.543k; tokens: 2.475977M; dataset ratio: [0.8096529168081933, 0.1903470831918067](vocab) Float32[0.97593045, 0.024069577](token)
Profile size: 74.684k(3.0%) @freq=101.0; [min(169017, 74684)]


(Pair{Vector{UInt8}, Float32}[[0x20] => 5.7489185, [0x61] => 2.946159, [0x65] => 2.6331801, [0x74] => 1.8507848, [0x6e] => 1.8268623, [0x69] => 1.8056064, [0x72] => 1.2130071, [0x64] => 1.1891679, [0x6e, 0x20] => 1.016678, [0x73] => 1.0038853  …  [0x6e, 0x20, 0x69, 0xe1, 0xb9, 0x9b] => 0.00023525237, [0x75, 0x66, 0x66, 0xc9, 0xa3, 0x61] => 0.00023525237, [0x75, 0x77, 0x65, 0xe1, 0xb8, 0x8d] => 0.00023525237, [0x75, 0xc9, 0xa3, 0x75, 0x72, 0x20] => 0.00023525237, [0x61, 0x73, 0x20, 0x6b, 0x72, 0x61, 0x20] => 0.00023525237, [0x64, 0x65, 0x67, 0x20, 0x74, 0x69, 0x72] => 0.00023525237, [0x73, 0x20, 0x6e, 0x20, 0x74, 0x61, 0x6c] => 0.00023525237, [0x74, 0x75, 0x66, 0x66, 0xc9, 0xa3, 0x61] => 0.00023525237, [0x75, 0x66, 0x66, 0xc9, 0xa3, 0x61, 0x20] => 0.00023525237, [0x75, 0x77, 0x65, 0xe1, 0xb8, 0x8d, 0x20] => 0.00023525237], Float32[29.956814, 29.393185, 28.828836, 28.257162, 27.657898, 26.988245, 26.211264])

In [13]:
sG[cp],sG[cp+1]

(UInt8[0x75, 0x77, 0x65, 0xe1, 0xb8, 0x8d, 0x20] => 0.00023525237f0, UInt8[0x73, 0x6e, 0x79] => 0.00023515218f0)

In [14]:
S[first(sG[cp])],S[first(sG[cp+1])]

(101.0f0, 9.0f0)

In [22]:
Ds[2][first(sG[cp])],Ds[2][first(sG[cp+1])]


(6.0f0, 9.0f0)

In [2]:
D,h = build_ngram_profile("zho")

# zho
total vocab: 4267.401k; tokens: 4.783494M;

 dataset ratio: [0.42586130556450996, 0.57413869443549](vocab) Float32[0.5644623, 0.4355377](token)
Profile size: 100.0k(2.0%) @freq=25.0; [min(1697115, 216718)]


(Float32[28.993256, 28.527767, 28.056707, 27.518705, 26.87696, 26.11319, 25.23846], Pair{Vector{UInt8}, Float32}[[0xe5] => 2.6759343, [0xe6] => 1.8406507, [0x20] => 1.751646, [0xe4] => 1.4011028, [0xe7] => 1.2207347, [0xe8] => 1.006906, [0xe9] => 0.70510626, [0xb8] => 0.6879991, [0xba] => 0.62958765, [0x88] => 0.5611001  …  [0xe5, 0x80, 0x8b, 0xe5, 0xad, 0xb8] => 0.00016786745, [0xe5, 0x8d, 0x81, 0xe5, 0xa4, 0x9a] => 0.00016786745, [0xe5, 0xa5, 0xb9, 0xe8, 0xbf, 0x98] => 0.00016786745, [0xe5, 0xb0, 0x8f, 0xe6, 0x8f, 0x90] => 0.00016786745, [0xe5, 0xba, 0x95, 0xe4, 0xb8, 0x8b] => 0.00016786745, [0xe6, 0x89, 0x80, 0xe8, 0xb0, 0x93] => 0.00016786745, [0xe8, 0xbf, 0x98, 0xe8, 0xa6, 0x81] => 0.00016786745, [0xe9, 0x80, 0x99, 0xe5, 0xb0, 0xb1] => 0.00016786745, [0x20, 0xe4, 0xbc, 0x9a, 0xe8, 0xae, 0xae] => 0.00016786745, [0x20, 0xe9, 0x86, 0xab, 0xe7, 0x94, 0x9f] => 0.00016786745])

In [12]:
@time build_ngram_profiles(["eng"], "ngrams")

# eng


total vocab: 3257.069k; tokens: 4.090477M; dataset ratio: [0.7086161344997173, 0.2913838655002827](vocab) [0.9664233351915567, 0.0335766648084433](token)
Profile size: 100.0k(3.0%) @freq=588.0; [min(171588, 240298)]


171.573536 seconds (538.60 M allocations: 33.011 GiB, 34.52% gc time)


172.016047 seconds (541.21 M allocations: 33.499 GiB, 34.43% gc time)


In [2]:
@time build_ngram_profiles(LANGS, "ngrams")

# ara
total vocab: 338.742k; tokens: 0.495728M; dataset ratio: [0.45305691831004097, 0.546943081689959](vocab) Float32[0.5281706, 0.4718294](token)


Profile size: 27.978k(8.0%) @freq=90.0; [min(27978, 110975)]
 14.430601 seconds (46.14 M allocations: 3.474 GiB, 13.24% gc time, 13.42% compilation time)

# bel
total vocab: 289.809k; tokens: 0.387915M; dataset ratio: [0.30759058041065696, 0.692409419589343](vocab) Float32[0.24278557, 0.7572144](token)


Profile size: 21.159k(7.0%) @freq=127.0; [min(21159, 59116)]
 10.977598 seconds (36.07 M allocations: 2.851 GiB, 15.18% gc time)


# ben
total vocab: 119.452k; tokens: 0.142487M; dataset ratio: [0.16760125485131977, 0.8323987451486803](vocab) Float32[0.1589021, 0.8410979](token)

...

# yid
total vocab: 143.749k; tokens: 0.19603M; dataset ratio: [0.3226393919298067, 0.6773606080701934](vocab) Float32[0.27865636, 0.72134364](token)


Profile size: 10.043k(7.0%) @freq=125.0; [min(10043, 41927)]
  7.620894 seconds (33.26 M allocations: 2.650 GiB, 20.07% gc time)


# zho
total vocab: 4264.385k; tokens: 4.779538M; dataset ratio: [0.426213788864112, 0.573786211135888](vocab) Float32[0.5649874, 0.4350126](token)


Profile size: 100.0k(2.0%) @freq=21.0; [min(1697106, 216515)]
 22.586473 seconds (39.64 M allocations: 3.728 GiB, 22.55% gc time)

1703.548984 seconds (5.51 G allocations: 675.745 GiB, 17.66% gc time, 0.13% compilation time)


In [3]:
@time build_ngram_profiles(LANGS[end-1:end], "ngrams")

# yid


total vocab: 135.062k; tokens: 0.1862M; dataset ratio: [0.33967239527389903, 0.660327604726101](vocab) [0.32930428350561436, 0.6706957164943856](token)
Profile size: 10.008k(7.0%) @freq=162.0; [min(10008, 41673)]
  7.859272 seconds (29.87 M allocations: 2.480 GiB, 12.81% gc time)


# zho


total vocab: 3838.139k; tokens: 4.300675M; dataset ratio: [0.4736709935068332, 0.5263290064931667](vocab) [0.6262618895235783, 0.3737381104764217](token)
Profile size: 100.0k(3.0%) @freq=17.0; [min(1586276, 172656)]


 21.605402 seconds (36.92 M allocations: 3.748 GiB, 22.87% gc time)


 29.862584 seconds (69.11 M allocations: 6.511 GiB, 20.30% gc time)


In [8]:
include("build_profiles.jl")


build_all_ngrams_profiles (generic function with 1 method)

In [11]:
h,D = build_ngram_profile("ben")

# ben
total vocab: 119.452k; tokens: 0.142487M; dataset ratio: [0.16760125485131977, 0.8323987451486803](vocab) Float32[0.1589021, 0.8410979](token)


Profile size: 2.917k(2.0%) @freq=1136.0; [min(2917, 17844)]


(Float32[29.24378, 29.05927, 28.875776, 28.695269, 28.51558, 28.334982, 28.154366], Pair{Vector{UInt8}, Float32}[[0xa6] => 7.969062, [0xe0] => 7.731554, [0xe0, 0xa6] => 7.722473, [0x20] => 5.974077, [0x20, 0xe0] => 5.7742343, [0x20, 0xe0, 0xa6] => 5.7696047, [0xb0] => 1.035716, [0xa6, 0xb0] => 1.0354671, [0xe0, 0xa6, 0xb0] => 1.0354671, [0xb0, 0x20] => 0.85204685  …  [0xa6, 0x20, 0xe0, 0xa6, 0x95, 0x20] => 0.006181228, [0xa6, 0xa6, 0x20, 0xe0, 0xa6, 0x95, 0x20] => 0.006181228, [0xa6, 0xb8, 0xe0, 0xa6, 0xb9] => 0.006178718, [0xe0, 0xa6, 0xb8, 0xe0, 0xa6, 0xb9] => 0.006178718, [0x9f, 0x20, 0xe0, 0xa6, 0xaa, 0x20] => 0.0061763423, [0x9f, 0x20, 0xe0, 0xa6, 0xaa, 0x20, 0xe0] => 0.0061763423, [0xa6, 0x9f, 0x20, 0xe0, 0xa6, 0xaa, 0x20] => 0.0061763423, [0xb2, 0x20, 0xe0, 0xa6, 0x86] => 0.0061759157, [0xa6, 0xb2, 0x20, 0xe0, 0xa6, 0x86] => 0.0061759157, [0xe0, 0xa6, 0xb2, 0x20, 0xe0, 0xa6, 0x86] => 0.0061759157])

In [38]:
dump_ngram_table(h, D, "nt.txt", compress_level=0)

In [39]:
h2, D2 = load_ngram_table("nt.txt")

(Float32[28.294203, 28.028467, 27.762024, 27.495188, 27.227724, 26.957682, 26.680351], Any[UInt8[0xd7] => 12.22566f0, UInt8[0x20] => 3.8148732f0, UInt8[0x20, 0xd7] => 3.5415354f0, UInt8[0x99] => 1.5719957f0, UInt8[0xd7, 0x99] => 1.5719957f0, UInt8[0x90] => 1.4738423f0, UInt8[0xd7, 0x90] => 1.4738423f0, UInt8[0x99, 0xd7] => 1.3780934f0, UInt8[0xd7, 0x99, 0xd7] => 1.3780934f0, UInt8[0xa2] => 1.1610758f0  …  UInt8[0xd7, 0xa2, 0xd7, 0x9d, 0x20, 0xd7, 0xa1] => 0.0016291593f0, UInt8[0x95, 0xd7, 0x96, 0xd7, 0x90] => 0.0016289278f0, UInt8[0xd7, 0x95, 0xd7, 0x96, 0xd7, 0x90] => 0.0016289278f0, UInt8[0x97, 0xd7, 0x94, 0x20, 0xd7] => 0.0016286962f0, UInt8[0xd7, 0x97, 0xd7, 0x94, 0x20, 0xd7] => 0.0016286962f0, UInt8[0x20, 0xd7, 0xa9, 0xd7, 0x9e, 0xd7, 0x95] => 0.001628505f0, UInt8[0x93, 0xd7, 0x96, 0xd7, 0xa2] => 0.0016281932f0, UInt8[0x93, 0xd7, 0x96, 0xd7, 0xa2, 0xd7] => 0.0016281932f0, UInt8[0xd7, 0x93, 0xd7, 0x96, 0xd7, 0xa2] => 0.0016281932f0, UInt8[0xd7, 0x93, 0xd7, 0x96, 0xd7, 0xa2, 0xd7] =

In [40]:
D==D2, h==h2

(true, true)

In [16]:
for cl in 63:-1:0
    println(cl)
    @time dump_ngram_table(h, D, "ct3/$(string(cl, pad=2)).txt", compress_level=cl)
    h2, D2 = load_ngram_table("ct3/$(string(cl, pad=2)).txt")
    @assert D==D2 && h==h2
end

63


  0.304139 seconds (1.22 M allocations: 356.017 MiB, 15.44% gc time)
62


  0.298373 seconds (1.20 M allocations: 350.491 MiB, 13.30% gc time)
61


  0.325974 seconds (1.18 M allocations: 344.960 MiB, 14.94% gc time)
60


  0.298288 seconds (1.17 M allocations: 339.423 MiB, 14.86% gc time)
59


...

  0.056019 seconds (254.01 k allocations: 59.628 MiB, 9.79% gc time)
9
  0.062403 seconds (235.37 k allocations: 53.971 MiB, 7.27% gc time)
8


  0.050347 seconds (216.76 k allocations: 48.305 MiB, 9.80% gc time)
7
  0.054194 seconds (198.24 k allocations: 42.648 MiB, 11.11% gc time)
6


  0.040564 seconds (179.60 k allocations: 36.984 MiB, 14.27% gc time)
5
  0.040295 seconds (160.86 k allocations: 31.301 MiB, 20.81% gc time)
4


  0.028306 seconds (142.05 k allocations: 25.629 MiB, 21.63% gc time)
3
  0.020315 seconds (122.98 k allocations: 19.945 MiB)
2
  0.024066 seconds (103.89 k allocations: 14.216 MiB, 21.12% gc time)
1


  0.007896 seconds (84.26 k allocations: 8.496 MiB)
0
  0.004130 seconds (59.59 k allocations: 2.674 MiB)


In [13]:
`ls -l ct3`|>run

总计 1756
-rw-rw-r-- 1 guoyongzhi guoyongzhi 52287  8月 24 13:35 00.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 26125  8月 24 13:35 01.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 26042  8月 24 13:35 02.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 25982  8月 24 13:35 03.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 25851  8月 24 13:35 04.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 25784  8月 24 13:35 05.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 25735  8月 24 13:35 06.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 25655  8月 24 13:35 07.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 25570  8月 24 13:35 08.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 25491  8月 24 13:35 09.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 25453  8月 24 13:35 10.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 25399  8月 24 13:35 11.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 25359  8月 24 13:35 12.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 25325  8月 24 13:35 13.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 25279  8月 24 13:35 14.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 25241  8月 24 13:35 15.txt
-rw-rw-r-- 1 guo

Process(`ls -l ct3`, ProcessExited(0))

In [15]:
`ls -l ct2`|>run

总计 57504
-rw-rw-r-- 1 guoyongzhi guoyongzhi 1260021  8月 24 13:32 00.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 1031312  8月 24 13:32 01.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 1015207  8月 24 13:32 02.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi 1003574  8月 24 13:32 03.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi  994241  8月 24 13:32 04.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi  986098  8月 24 13:32 05.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi  979475  8月 24 13:32 06.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi  973532  8月 24 13:32 07.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi  967439  8月 24 13:32 08.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi  962089  8月 24 13:32 09.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi  957334  8月 24 13:32 10.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi  952973  8月 24 13:32 11.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi  948939  8月 24 13:31 12.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi  945096  8月 24 13:31 13.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi  941344  8月 24 13:31 14.txt
-rw-rw-r-- 1 guoyongzhi guoyongzhi  937580  8月

Process(`ls -l ct2`, ProcessExited(0))

In [21]:
lcs_unzip(str, refer)

"kkabskk"

In [107]:
string('a', "ff")

"aff"

In [110]:
length(1:0)

0

In [111]:
match(r"abdc", "ksabcab")

In [112]:
Char(Int('A') + 26 - 1)

'Z': ASCII/Unicode U+005A (category Lu: Letter, uppercase)

In [113]:
Char.(Int('A') .+ findfirst("abc", "ksabcab"))

3-element Vector{Char}:
 'D': ASCII/Unicode U+0044 (category Lu: Letter, uppercase)
 'E': ASCII/Unicode U+0045 (category Lu: Letter, uppercase)
 'F': ASCII/Unicode U+0046 (category Lu: Letter, uppercase)

In [114]:
splice!

splice! (generic function with 9 methods)

In [115]:
D2

10008-element Vector{Pair{Vector{UInt8}, Float64}}:
                                     [0xd7] => 12.132796741701616
                                     [0x20] => 3.789403273495392
                               [0x20, 0xd7] => 3.5162187256097748
                                     [0x99] => 1.5591632268892548
                               [0xd7, 0x99] => 1.5591632268892548
                                     [0x90] => 1.4649511189375592
                               [0xd7, 0x90] => 1.4649511189375592
                               [0x99, 0xd7] => 1.3667675852862382
                         [0xd7, 0x99, 0xd7] => 1.3667675852862382
                                     [0xa2] => 1.1506468924279525
                                            ⋮
       [0x99, 0xd7, 0x96, 0xd7, 0x9d, 0x20] => 0.0016153944909111575
       [0xd7, 0x99, 0xd7, 0x96, 0xd7, 0x9d] => 0.0016153944909111575
 [0xd7, 0x99, 0xd7, 0x96, 0xd7, 0x9d, 0x20] => 0.0016153944909111575
 [0x95, 0xd7, 0x99, 0xd7, 0x99, 0xd7

In [116]:
G = D
sort(collect(G), by=i->(-last(i), length(first(i)), first(i)))

10008-element Vector{Pair{Vector{UInt8}, Float64}}:
                                     [0xd7] => 12.132796741701616
                                     [0x20] => 3.789403273495392
                               [0x20, 0xd7] => 3.5162187256097748
                                     [0x99] => 1.5591632268892548
                               [0xd7, 0x99] => 1.5591632268892548
                                     [0x90] => 1.4649511189375592
                               [0xd7, 0x90] => 1.4649511189375592
                               [0x99, 0xd7] => 1.3667675852862382
                         [0xd7, 0x99, 0xd7] => 1.3667675852862382
                                     [0xa2] => 1.1506468924279525
                                            ⋮
       [0x99, 0xd7, 0x96, 0xd7, 0x9d, 0x20] => 0.0016153944909111575
       [0xd7, 0x99, 0xd7, 0x96, 0xd7, 0x9d] => 0.0016153944909111575
 [0xd7, 0x99, 0xd7, 0x96, 0xd7, 0x9d, 0x20] => 0.0016153944909111575
 [0x95, 0xd7, 0x99, 0xd7, 0x99, 0xd7

In [117]:
build_ngram_profile("yid")

# yid


total vocab: 135.062k; tokens: 0.1862M; dataset ratio: [0.33967239527389903, 0.660327604726101](vocab) [0.32930428350561436, 0.6706957164943856](token)
Profile size: 10.008k(7.0%) @freq=162.0; [min(10008, 41673)]


(Pair{Vector{UInt8}, Float64}[[0xd7] => 12.132796741701616, [0x20] => 3.789403273495392, [0x20, 0xd7] => 3.5162187256097748, [0x99] => 1.5591632268892548, [0xd7, 0x99] => 1.5591632268892548, [0x90] => 1.4649511189375592, [0xd7, 0x90] => 1.4649511189375592, [0x99, 0xd7] => 1.3667675852862382, [0xd7, 0x99, 0xd7] => 1.3667675852862382, [0xa2] => 1.1506468924279525  …  [0x99, 0xd7, 0x96, 0xd7, 0x9d] => 0.0016153944909111575, [0x99, 0xd7, 0x96, 0xd7, 0x9d, 0x20] => 0.0016153944909111575, [0xd7, 0x99, 0xd7, 0x96, 0xd7, 0x9d] => 0.0016153944909111575, [0xd7, 0x99, 0xd7, 0x96, 0xd7, 0x9d, 0x20] => 0.0016153944909111575, [0x95, 0xd7, 0x99, 0xd7, 0x99, 0xd7, 0x9c] => 0.0016152803763646896, [0x90, 0xd7, 0x9b, 0xd7, 0x98, 0xd7, 0xa2] => 0.001614766205078347, [0xa9, 0xd7, 0x98, 0xd7, 0x90, 0xd7, 0xa8] => 0.001613737862505662, [0xa2, 0x20, 0xd7, 0x93, 0xd7, 0x99, 0x20] => 0.0016132236912193194, [0x93, 0xd7, 0x96, 0xd7, 0xa2, 0xd7, 0xa8] => 0.0016126407888742012, [0x9c, 0xd7, 0x99, 0xd7, 0x93] => 0.0

In [118]:
build_ngram_profile("zho")
build_ngram_profile("yid")
build_ngram_profile("ina")


# zho


total vocab: 3838.139k; tokens: 4.300675M; dataset ratio: [0.4736709935068332, 0.5263290064931667](vocab) [0.6262618895235783, 0.3737381104764217](token)
Profile size: 100.0k(3.0%) @freq=17.0; [min(1586276, 172656)]


# yid

In [ ]:
build_ngram_profile("zho", ngram=5)
build_ngram_profile("yid", ngram=5)
build_ngram_profile("ina", ngram=5)
;

In [ ]:
build_ngram_profile("zho", ratio=0.7)
build_ngram_profile("yid", ratio=0.7)
build_ngram_profile("ina", ratio=0.7)


In [ ]:
build_ngram_profile("zho", ratio=0.7, ngram=5)
build_ngram_profile("yid", ratio=0.7, ngram=5)
build_ngram_profile("ina", ratio=0.7, ngram=5)
